# Monitors Debugger Notebook - Verify Monitors Setup and Execution (Optionally)

In [14]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import io
import random
import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

from threading import Timer

CPU times: user 128 μs, sys: 0 ns, total: 128 μs
Wall time: 133 μs


In [15]:
sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

# Retrieve the default Amazon S3 bucket associated with the SageMaker session.
bucket = sagemaker_session.default_bucket()
print("Bucket:", bucket)

# Get the IAM role associated with the current SageMaker notebook or environment.
role = get_execution_role()
print("RoleArn:", role)

# Get the AWS region name for the current session.
region = boto3.Session().region_name
print("Region", region)

# Retrieve the AWS account ID of the caller using the Security Token Service (STS) client.
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Boto3 client for the SageMaker service, specifying the AWS region.
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Create an S3 client
s3 = boto3.client('s3')

Bucket: sagemaker-us-east-1-084375567266
RoleArn: arn:aws:iam::084375567266:role/service-role/SageMaker-ExecutionRole-20250222T162355
Region us-east-1


In [19]:
!aws sagemaker list-monitoring-schedules --endpoint-name "retain-ai-xgb-endpoint"

{
    "MonitoringScheduleSummaries": []
}


In [18]:
# !aws sagemaker delete-monitoring-schedule --monitoring-schedule-name "monitoring-schedule-2025-02-24-04-07-06-105"


An error occurred (ResourceNotFound) when calling the DeleteMonitoringSchedule operation: Monitoring Schedule arn:aws:sagemaker:us-east-1:084375567266:monitoring-schedule/monitoring-schedule-2025-02-24-04-07-06-105 not found


In [23]:
!aws s3 ls s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/ --recursive

2025-02-24 05:33:32       2274 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/04/3331.jsonl
2025-02-24 05:34:32      11388 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3431.jsonl
2025-02-24 05:35:32      13660 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3531.jsonl
2025-02-24 05:36:33      13660 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3631.jsonl
2025-02-24 05:37:33      13657 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3732.jsonl
2025-02-24 05:38:33      13662 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3832.jsonl
2025-02-24 05:39:33      13661 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3932.jsonl
2025-02-24 05:40:33      11380 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/4032.jsonl
2025-02-24 05:41:33      13657 aai-540-group-3-final-pro